In [2]:
from PIL  import ImageGrab as IG
from core import connect   as connect

import serial    as     sr
import time      as     tm
import numpy     as     np
import threading as     tr
import os

In [10]:
WDseq(10,25)

withdrawing 10 mL, at 25 ml/min


In [12]:
conn.stopPump()

['stop', 'Pump stop!', '>']

    Sensor  Width: 288
    Sensor Height: 375
    Offset      x: 768
    Offset      y: 634

In [2]:
# framing for image capture
i1 = 305
i2 = 100
i3 = i1 + 600
i4 = i2 + 885

expName = 'phi20_L200um_d50um_a1.5mm_take2'
os.mkdir(expName)

def peek(fileNameTime):
    sc1 = IG.grab(bbox=(i1, i2, i3, i4))
    imageName = expName+'/' + str(fileNameTime) + '.png'
    sc1.save(imageName)
    sc1.close()

In [3]:
busy = False 

In [3]:
# get open port info
portinfo = connect.getOpenPorts() 

# 'windows' or 'mac'
deviceType = 'windows'

# MUST set baudrate in pump "System Settings", and MUST match this rate:
baudratePUMP = 9600
baudrateINO  = 38400

if deviceType == 'mac':
    print('macDevice')
    # initiate Connection object with first open port
    conn = connect.Connection(port = '/dev/tty.usbserial-A10MLXX5', baudrate = baudratePUMP, x = 0, mode = 0,verbose = False)

    arduino = sr.Serial(port='/dev/tty.usbmodem2101', baudrate=baudrateINO, timeout=1)
    tm.sleep(2)  # Give some time for the connection to establish
elif deviceType == 'windows':
    print('windowsDevice')
    # initiate Connection object with first open port
    conn = connect.Connection(port = 'COM3', baudrate = baudratePUMP, x = 0, mode = 0,verbose = False)

    arduino = sr.Serial(port='COM7', baudrate=baudrateINO, timeout=1)
    tm.sleep(2)  # Give some time for the connection to establish

windowsDevice


In [5]:
def WDseq(v,r):
    print("withdrawing", v, "mL, at", r,"ml/min")

    # Open Connection to pump
    conn.openConnection()

    conn.stopPump()

    # Setup parameters for basic run
    units    = 'mL/min'		 	# OPTIONS: 'mL/min','mL/hr','μL/min','μL/hr'
    diameter = 26.8             # 28.6mm diameter - can be set in pump GUI
    volume   = -v              # withdraw 10 mL volume 
    rate     = r                # 5 mL/min flow rate
    runtime  = volume/rate      # this is calculated implictly by pump
    delay    = 0                # 1 is 60 second delay

    # Communicate parameters to pump
    conn.setUnits(units)
    conn.setDiameter(diameter)  
    conn.setVolume(volume)      
    conn.setRate(rate)          
    conn.setDelay(delay)       
    tm.sleep(1)

    # Start pump
    conn.startPump()

In [6]:
def INseq(v,r):
    print("infusing", v, "mL, at", r,"ml/min")
    # Open Connection to pump
    conn.openConnection()

    conn.stopPump()

    # Setup parameters for basic run
    units    = 'mL/min'		 	# OPTIONS: 'mL/min','mL/hr','μL/min','μL/hr'
    diameter = 26.8             # 28.6mm diameter - can be set in pump GUI
    volume   = v              # withdraw 10 mL volume 
    rate     = r                # 5 mL/min flow rate
    runtime  = volume/rate      # this is calculated implictly by pump
    delay    = 0                # 1 is 60 second delay

    # Communicate parameters to pump
    conn.setUnits(units)
    conn.setDiameter(diameter)  
    conn.setVolume(volume)      
    conn.setRate(rate)          
    conn.setDelay(delay)    
       
    tm.sleep(1)

    # Start pump
    conn.startPump()

In [7]:
def unClogSeq(v,r):
    print("begin unclogging")
    global busy
    global running
    
    pump_buffer_time = 2.5
    slowInFactor = 0.25
    flushVolume = 2
    # big infuse
    INseq(v,r)
    tm.sleep(v / r * 60 + pump_buffer_time)

    # cycle 1
    WDseq(flushVolume,r)
    tm.sleep(flushVolume / r * 60 + pump_buffer_time)
    INseq(flushVolume,r * slowInFactor)
    tm.sleep(flushVolume / r / slowInFactor * 60 + pump_buffer_time)

    # # cycle 2
    # WDseq(2.5,r)
    # tm.sleep(2.5 / r * 60 + pump_buffer_time)
    # INseq(2.5,r)
    # tm.sleep(2.5 / r * 60 + pump_buffer_time)

    # # cycle 3
    # WDseq(5,r)
    # tm.sleep(5 / r * 60 + pump_buffer_time)
    # INseq(5,r)
    # tm.sleep(5 / r * 60 + pump_buffer_time)
   
    busy    = False
    running = False

In [8]:
def isClog(pressure, clogLim, vinfed):
    global busy
    global t_clog
    global start_time


In [9]:
def runExp(v,r):
    # Open Connection to pump
    conn.openConnection()

    conn.stopPump()

    # Setup parameters for basic run
    units    = 'mL/min'		 	# OPTIONS: 'mL/min','mL/hr','μL/min','μL/hr'
    diameter = 26.8             # 28.6mm diameter - can be set in pump GUI
    volume   = -v               # withdraw 10 mL volume 
    rate     = r                # 5 mL/min flow rate
    runtime  = volume/rate      # this is calculated implictly by pump
    delay    = 0                # 1 is 60 second delay

    # Communicate parameters to pump
    conn.setUnits(units)
    conn.setDiameter(diameter)  
    conn.setVolume(volume)      
    conn.setRate(rate)          
    conn.setDelay(delay)       

    tm.sleep(1)
    # Start pump
    conn.startPump()

In [10]:
# def read_arduino():
#     try:
#         arduino.flushInput()
#         data = arduino.readline()#.decode('utf-8').strip()
#         if data and data.isdigit():
#             return int(data)
#         else:
#             print("here")
#             return None
#     except:
#         # Catch any exception and ignore it
#         print("this here")
#         return None

def read_arduino():
    try:
        arduino.flushInput()  # Clear any junk in the input buffer
        data = arduino.readline().decode('utf-8').strip()
        return int(data)
    except:
        return None


In [11]:
# Function to read from Arduino in a separate thread
def read_arduino_thread():
    global latest_value
    while True:
        value = read_arduino()  # Your function to read from Arduino
        if value is not None:
            latest_value = value
        tm.sleep(0.25)  # Sleep to match Arduino sampling rate

In [12]:
import threading
import time as tm
import numpy as np

global running
global t_clog
global start_time
global latest_value

running = False
busy = False

data_list = []
time_list = []
time_list_1 = [0]
pressureOfClog = 93 

t_clog = []
latest_value = None

exp_run_rate = 0.5 # ml/min
exp_vol      = 30 # ml
start_time   = tm.time()


# Start the Arduino reading thread
thread = threading.Thread(target=read_arduino_thread)
thread.daemon = True  # Daemon thread will automatically close when the main program exits
thread.start()

print(expName)
try:
    while True:
        value = latest_value
         # print("value is", value, " time is ", tm.time()-start_time)
        tm.sleep(0.1)
        
        if value is not None and value > 85:
            if not running and not busy:
                running = True
                runExp(v = exp_vol, r = exp_run_rate)
                start_time_clog = tm.time()
                print("running exp for volume", exp_vol, " with flow rate of", exp_run_rate)

            time_running = tm.time() - start_time
            P = value
            cL = pressureOfClog

            if P <= cL and not busy:
                t_clog     = tm.time() - start_time_clog
                round_time = np.round(tm.time() - start_time, 0)
 
                vinfed = exp_run_rate / 60 * t_clog
                print("pressure fell to ", P, "which is below clog limit of" , cL)
                print("Time to clog is ", t_clog)
                busy = True
                
                peek(round_time) # take image of clog
                
                unClogSeq(v=vinfed, r=50)

                time_list_1.append(t_clog)


except KeyboardInterrupt:
    print("Exiting...")
    conn.stopPump()
    


print("Data collection complete.")


phi20_L200um_d50um_a1.5mm_take2
running exp for volume 30  with flow rate of 0.5
pressure fell to  93 which is below clog limit of 93
Time to clog is  689.2987179756165
begin unclogging
infusing 5.744155983130137 mL, at 50 ml/min
withdrawing 2 mL, at 50 ml/min
infusing 2 mL, at 12.5 ml/min
running exp for volume 30  with flow rate of 0.5
pressure fell to  93 which is below clog limit of 93
Time to clog is  536.1978144645691
begin unclogging
infusing 4.468315120538076 mL, at 50 ml/min
withdrawing 2 mL, at 50 ml/min
infusing 2 mL, at 12.5 ml/min
running exp for volume 30  with flow rate of 0.5
pressure fell to  89 which is below clog limit of 93
Time to clog is  835.9427499771118
begin unclogging
infusing 6.966189583142598 mL, at 50 ml/min
withdrawing 2 mL, at 50 ml/min
infusing 2 mL, at 12.5 ml/min
running exp for volume 30  with flow rate of 0.5
pressure fell to  91 which is below clog limit of 93
Time to clog is  1040.0116035938263
begin unclogging
infusing 8.666763363281886 mL, at 50

In [ ]:
print(time_list_1)

[0, 4.016052722930908, 0.10051298141479492, 11.450860023498535, 13.758227109909058, 15.563290119171143, 12.648094892501831, 12.754050254821777, 14.656028032302856, 16.66805076599121, 11.549072265625, 12.652039289474487, 13.556859731674194, 17.667781352996826, 15.661645412445068, 19.58163547515869, 19.483900785446167, 9.5417959690094, 10.645823001861572, 15.663398504257202, 14.55629587173462, 16.667083501815796, 14.659518003463745, 14.555873155593872, 14.654400825500488, 8.631706953048706, 13.757527828216553, 8.633388042449951, 13.658949136734009, 16.56506657600403, 12.551541090011597, 33.74010348320007, 19.380653619766235, 11.646848917007446, 11.749492645263672, 7.629624843597412, 11.747594833374023, 16.669146299362183, 14.660993099212646, 18.577792167663574, 14.66231393814087, 16.56752061843872, 23.591063976287842, 16.56912350654602, 0.10073971748352051, 0.10138368606567383, 1.9079232215881348, 3.8174047470092773, 0.0, 7.430898904800415, 0.0010466575622558594, 0.1012413501739502, 0.0,

In [ ]:
global running
global t_clog
global start_time

running = False

data_list = []
time_list = []
pressureOfClog = 90 

t_clog = []

exp_run_rate = 5 # ml/min
exp_vol      = 30 # ml
start_time   = tm.time()

try:
    while True:
        value = read_arduino()
        print("value is", value, " time is ", tm.time()-start_time)
        tm.sleep(0.1)
        if value is not None and value > 85:
            if running is not True and not busy:
                running = True
                runExp(v = exp_vol, r = exp_run_rate)
                start_time_clog = tm.time()
                print("running exp for volume", exp_vol, " with flow rate of", exp_run_rate)

            time_running = tm.time()-start_time
            #time_list.append(time_running)
            #data_list.append(value)
            #pressmBar = (value - 100) * 2000/398
            

            # isClog(pressure = value, clogLim = pressureOfClog, vinfed= volume)
            P = value
            cL = pressureOfClog

            # print("pressure is ", P)
            # print("clog lim is ", cL)
            if P <= cL and not busy:
                t_clog = tm.time() - start_time_clog 
                vinfed = exp_run_rate / 60 * t_clog
                print("pressure fell to ", P, "which is below clog limit of" , cL)
                print("Time to clog is ", t_clog)
                busy = True
                unClogSeq(v=vinfed,r=50)

            # Print the latest value every few seconds (e.g., every 5 seconds)
            round_time = np.round(tm.time() - start_time,0)
            # if round_time % 3 == 0:  # 0.1s * 30 = 3 seconds
            #     print(f"Latest value (after {round_time} seconds): {value}")
            #     # print("press mbar is", pressmBar)
        
except KeyboardInterrupt:
    print("Exiting...")
    # Handle any cleanup if necessary here

print("Data collection complete.")


In [13]:
time_list_1

[0,
 689.2987179756165,
 536.1978144645691,
 835.9427499771118,
 1040.0116035938263,
 1733.2706897258759,
 58.46311712265015,
 1288.9169380664825,
 1513.5819990634918,
 824.0536532402039,
 2556.723073244095,
 1357.9086894989014,
 1936.16370511055,
 2407.6668231487274,
 2154.8462946414948,
 1227.9055387973785,
 1310.0400862693787,
 1881.6865298748016,
 1136.4693348407745,
 2447.6389336586,
 2366.6297500133514,
 1750.975837945938,
 2870.729944705963,
 1040.7150259017944,
 2418.504963159561,
 1181.2745690345764,
 2448.1853053569794,
 2710.617676258087,
 1910.7134110927582,
 1566.4812715053558,
 1474.912941455841,
 1810.820639371872]

In [ ]:
round_time

57957.0